In [1]:
pip install opencv-python numpy matplotlib scikit-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /home/anaconda/.conda/envs/jupyter/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

# Function to preprocess images using OpenCV
def preprocess_image(image_path):
    # Load image in BGR format
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError(f"Image not found or unable to load: {image_path}")
    
    # Resize image
    image = cv2.resize(image, (200, 200))
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Perform Gaussian blur on CPU
    kernel_size = 5
    denoised_image = cv2.GaussianBlur(
        src=gray_image, ksize=(kernel_size, kernel_size), sigmaX=0
    )
    
    # Flatten the image
    denoised_image = denoised_image.flatten()
    return denoised_image

# Load dataset
def load_dataset(data_dir):
    features = []
    labels = []
    class_names = os.listdir(data_dir)
    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            features.append(preprocess_image(image_path))
            labels.append(label)
    return np.array(features), np.array(labels), class_names

# Dataset directory
data_dir = "./dataset"  # Replace with the actual dataset path

# Load and preprocess data
print("Loading data")
features, labels, class_names = load_dataset(data_dir)
print("Done Loading data")

# Split dataset
print("Splitting data")
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42
)
print("Done Splitting data")

# Initialize and train the Random Forest model
print("Init Model")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=56)
print("Fitting Model")
rf_model.fit(train_features, train_labels)
print("Done Fitting Model")

# Predict on training and test sets
print("Predicting")
train_preds = rf_model.predict(train_features)
test_preds = rf_model.predict(test_features)

# Compute accuracy
print("Accuracy Calculation")
train_accuracy = accuracy_score(train_labels, train_preds)
test_accuracy = accuracy_score(test_labels, test_preds)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(test_labels, test_preds, target_names=class_names))

# Confusion matrix
conf_matrix = confusion_matrix(test_labels, test_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)
disp.plot(cmap="Blues", xticks_rotation="vertical")
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix_rf.png")
print("Confusion matrix saved as 'confusion_matrix_rf.png'")
plt.show()

# Feature importance
importances = rf_model.feature_importances_
indices = np.argsort(importances)[-10:]  # Top 10 important features

plt.figure(figsize=(10, 5))
plt.barh(range(len(indices)), importances[indices], align='center')
plt.yticks(range(len(indices)), [f"Feature {i}" for i in indices])
plt.xlabel("Relative Importance")
plt.title("Top 10 Feature Importances")
plt.savefig("feature_importances_rf.png")
print("Feature importance plot saved as 'feature_importances_rf.png'")
plt.show()

Loading data
Done Loading data
Splitting data
Done Splitting data
Init Model
Fitting Model
Done Fitting Model
Predicting
Accuracy Calculation
Train Accuracy: 1.0000
Test Accuracy: 0.7108

Classification Report:
              precision    recall  f1-score   support

        Real       0.70      0.88      0.78     27184
          AI       0.75      0.48      0.59     20255

    accuracy                           0.71     47439
   macro avg       0.72      0.68      0.68     47439
weighted avg       0.72      0.71      0.70     47439

Confusion matrix saved as 'confusion_matrix_rf.png'


Feature importance plot saved as 'feature_importances_rf.png'


Class Imbalance:
The dataset appears imbalanced, with more samples for the "Real" class (27,184) than the "AI" class (20,255). This imbalance likely caused the model to bias predictions towards the "Real" class, leading to low recall for "AI."

High Recall for "Real" Class:
The model prioritizes identifying "Real" samples correctly, but at the cost of missing many "AI" samples.

Moderate Precision for "AI":
redictions for the "AI" class are moderately precise (75%), meaning when the model predicts "AI," it is often correct. However, many true "AI" samples remain unclassified.

In [ ]:
import joblib

# Export the trained model
joblib.dump(rf_model, 'full_random_forest_model.pkl')
print("Model saved as 'full_random_forest_model.pkl'")